In [ ]:
SELECT customer_id, FY, Auth_Date,Division, Department, Level1, Customer_seq,
DATE_DIFF(Auth_Date,Lag,DAY) AS Diff, RepeatPurchase
FROM (
SELECT customer_id, FY, Auth_Date,Division, Department, Level1, Customer_seq,
 LAG(Auth_Date) OVER (PARTITION BY customer_id ORDER BY  Auth_Date) AS Lag,
CASE WHEN customer_seq > 1 THEN 'Repeat Customer'
ELSE 'New Customer'
END AS RepeatPurchase
FROM (
    SELECT  *,
         CASE WHEN DATE(x.Auth_Date) >= '2019-04-01' AND DATE(x.Auth_Date) < '2020-04-01' THEN "FY20"
              WHEN DATE(x.Auth_Date) >= '2020-04-01' AND DATE(x.Auth_Date) < '2021-04-01' THEN "FY21"
              WHEN DATE(x.Auth_Date) >= '2021-04-01' AND DATE(x.Auth_Date) < '2022-04-01' THEN "FY22"
      END AS FY,
    RANK() OVER (PARTITION BY customer_id ORDER BY x.Auth_Date) AS customer_seq
    FROM
(
SELECT  
       DISTINCT customer_id,
        bh.bh_division AS Division, 
        bh.bh_reporting_department AS Department,
        bh.bh_level_1 AS Level1,
       DATE(o.order_authed_timestamp) AS Auth_Date,
        o.order_id,
        oi.order_item_sales AS SalesEx,
        oi.order_item_quantity AS Units
    FROM `tal-production-data-bi.tal_dm_order.orders_2010_2020` o,
        UNNEST(o.order_items) oi
    INNER JOIN `tal-production-data-bi.tal_dm_product.business_hierarchy` bh
        ON bh.product_id = oi.product_id
    WHERE   
        DATE(o.order_placed_timestamp) >= "2019-04-01" AND DATE(o.order_placed_timestamp) < "2020-01-01" 
        AND DATE(o.order_authed_timestamp) >= "2019-04-01" AND DATE(o.order_authed_timestamp) < "2020-01-01" 
        AND o.order_authorisation_status IN ('Auth')
        AND o.is_not_return_cancel_in_full IS TRUE
        #AND o.voucher_sales_percent =0
        AND is_test_customer = false
        AND o.order_authed_timestamp IS NOT NULL

UNION ALL

SELECT 
        DISTINCT customer_id,
        bh.bh_division AS Division, 
        bh.bh_reporting_department AS Department,
        bh.bh_level_1 AS Level1,
        DATE(o.order_authed_timestamp) AS Auth_Date,
        o.order_id,
        oi.order_item_sales AS SalesEx,
        oi.order_item_quantity AS Units
    FROM `tal-production-data-bi.tal_dm_order.orders_2020_2030` o,
        UNNEST(o.order_items) oi
    INNER JOIN `tal-production-data-bi.tal_dm_product.business_hierarchy` bh
        ON bh.product_id = oi.product_id
    WHERE   
        DATE(o.order_placed_timestamp) >= "2020-01-01" AND DATE(o.order_placed_timestamp) < "2022-06-01" 
        AND DATE(o.order_authed_timestamp) >= "2020-01-01" AND DATE(o.order_authed_timestamp) < "2022-06-01" 
        AND o.order_authorisation_status IN ('Auth')
        AND o.is_not_return_cancel_in_full IS TRUE
        #AND o.voucher_sales_percent =0
        AND is_test_customer = false
        AND o.order_authed_timestamp IS NOT NULL) x)
WHERE FY IS NOT NULL AND customer_id IN (8)
GROUP BY 1,2,3,4,5,6,7
ORDER BY Auth_Date ASC)